In [1]:
import os
%pwd

'c:\\Users\\SGU\\Projects\\mlops-with-mlflow\\research'

In [2]:
os.chdir(r"C:\Users\SGU\Projects\mlops-with-mlflow")
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [3]:
# creating entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from mlProject.utils.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARAMS_FILE_PATH
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
        print(self.config)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            config.root_dir,
            config.source_url,
            config.local_data_file,
            config.unzip_dir
        )
        
        

In [5]:
import os
import zipfile
from mlProject import logger
from mlProject.utils.common import getsize
from urllib import request

class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: {headers}")
        else:
            logger.info(f"file already exists with size: {getsize(self.config.local_data_file)}")
    
    def extract_zipfile(self):
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file) as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        
    


In [12]:
# pipeline

try:
    cfg_manager = ConfigurationManager()
    data_ingestion_config = cfg_manager.get_data_ingestion_config()
    print(data_ingestion_config)
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2023-08-10 16:41:04,736: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-10 16:41:04,736: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-10 16:41:04,736: INFO: common: yaml file: params.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artitacts/data_ingestion', 'source_url': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artitacts/data_ingestion/data.zip', 'unzip_dir': 'artitacts/data_ingestion'}, 'data_validation': {'root_dir': 'artitacts/data_validation'}, 'data_transformation': {'root_dir': 'artitacts/data_transformation'}}
[2023-08-10 16:41:04,744: INFO: common: created directory: artifacts]
[2023-08-10 16:41:04,747: INFO: common: created directory: artitacts/data_ingestion]
DataIngestionConfig(root_dir='artitacts/data_ingestion', source_url='https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', local_data_file='ar